## Modelling employee attrition

### Process:

- We are working with cleaned data
- Split into Train / Valid / Test datasets.
- 

In [17]:
import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost
import mlflow
import warnings

from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [2]:
import matplotlib as mpl
mpl.rcParams['font.family'] = 'sans-serif'

In [3]:
# Add my parent directory to path variables
current_location = Path(os.path.abspath('')).parent.resolve()
print(current_location)
sys.path.append(str(current_location))

/Users/ryanmccarlie/Documents/Problems/PwC Interview/Code


In [4]:
raw_data_dir = os.path.join(current_location, "raw_data")
os.makedirs(raw_data_dir, exist_ok=True)

transformed_data_dir = os.path.join(current_location, "transformed_data")
os.makedirs(transformed_data_dir, exist_ok=True)

output_data_dir = os.path.join(current_location, "output_data")
os.makedirs(output_data_dir, exist_ok=True)

processed_dir = os.path.join(current_location, "processed_dir")
os.makedirs(processed_dir, exist_ok=True)

models_dir = os.path.join(current_location, "models")
os.makedirs(models_dir, exist_ok=True)

report_dir = os.path.join(current_location, "reports")
os.makedirs(report_dir, exist_ok=True)

In [16]:
random_state_var = 42
random_state_var

42

In [5]:
def get_or_create_experiment(experiment_name: str) -> str:
    """
    Fetches the experiment with the given name if it exists. If it doesn't exist, creates a new one.
    
    Parameters:
    - experiment_name (str): The name of the MLFlow experiment.
    
    Returns:
    - experiment_id (str): The ID of the experiment to use.
    """
    mlflow.set_tracking_uri("file:///Users/ryanmccarlie/Documents/Problems/PwC%20Interview/Code/mlruns")

    # Fetch the experiment if it exists
    try:
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is not None:
            print(f"Experiment '{experiment_name}' exists with ID: {experiment.experiment_id}")
            return experiment.experiment_id
        else:
            # Create a new experiment if it does not exist
            experiment_id = mlflow.create_experiment(experiment_name)
            print(f"Created new experiment '{experiment_name}' with ID: {experiment_id}")
            return experiment_id
    except mlflow.MlflowException as e:
        print(f"Error: {e}")
        raise

In [6]:
# Example usage
experiment_name = "Employee_Attrition_Experiment"

# Get or create the experiment
experiment_id = get_or_create_experiment(experiment_name)

# Set the experiment ID for subsequent logging
mlflow.set_experiment(experiment_id=experiment_id)

Experiment 'Employee_Attrition_Experiment' exists with ID: 127905123878869483


<Experiment: artifact_location='file:///Users/ryanmccarlie/Documents/Problems/PwC%20Interview/Code/mlruns/127905123878869483', creation_time=1726585527884, experiment_id='127905123878869483', last_update_time=1726585527884, lifecycle_stage='active', name='Employee_Attrition_Experiment', tags={}>

## Read in transformed data

In [7]:
cleaned_fname = os.path.join(transformed_data_dir, "cleaned_scaled_data_17_09_2024.csv")
modelling_data = pd.read_csv(cleaned_fname)

modelling_data = modelling_data.drop(columns=["Attrition"])

# print(modelling_data.columns)
print(modelling_data.shape)
modelling_data.head(3)

(4310, 79)


,Age,YearsAtCompany,MonthlyIncome,DistanceFromHome,PercentSalaryHike,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,WorkLifeBalanceScore_dummy,RoleToCompanyRatio,...,MaritalGenderInteraction_Single_Female,MaritalGenderInteraction_Single_Male,OvertimeJobSatisfactionInteraction_No_1.0,OvertimeJobSatisfactionInteraction_No_2.0,OvertimeJobSatisfactionInteraction_No_3.0,OvertimeJobSatisfactionInteraction_No_4.0,OvertimeJobSatisfactionInteraction_Yes_1.0,OvertimeJobSatisfactionInteraction_Yes_2.0,OvertimeJobSatisfactionInteraction_Yes_3.0,OvertimeJobSatisfactionInteraction_Yes_4.0
0,1.094496,-0.882370,1.696610,-1.118965,1.645309,-0.866686,-0.667242,-0.927282,0.696054,-0.784017,...,False,False,True,False,False,False,False,False,False,False
1,0.356141,0.940307,-0.851763,1.000423,1.096108,0.820303,1.847459,0.825716,1.396495,0.051465,...,True,False,True,False,False,False,False,False,False,False
2,-0.677557,1.487110,-0.721991,-1.684135,-0.002294,0.398556,0.841578,-0.927282,-0.004388,-0.488790,...,False,True,False,False,False,True,False,False,False,False


## Train Test Split

In [9]:
# src/data_preprocessing.py
import pandas as pd
from sklearn.model_selection import train_test_split
import os

def split_and_save_data(input_df: pd.DataFrame, target_col: str, processed_dir:str, random_state=random_state_var):
    # Define features and target
    X = input_df.drop(target_col, axis=1)
    y = input_df[target_col]
    
    # Split into train (70%), validation (15%), and test (15%)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, random_state=random_state, stratify=y
    )
    X_valid, X_test, y_valid, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=random_state, stratify=y_temp
    )
    
    # Save splits
    train_df = pd.concat([X_train, y_train], axis=1)
    valid_df = pd.concat([X_valid, y_valid], axis=1)
    test_df = pd.concat([X_test, y_test], axis=1)
    
    train_df.to_csv(os.path.join(processed_dir, 'train.csv'), index=False)
    valid_df.to_csv(os.path.join(processed_dir, 'validation.csv'), index=False)
    test_df.to_csv(os.path.join(processed_dir, 'test.csv'), index=False)
    

    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_valid': X_valid,
        'y_valid': y_valid,
        'X_test': X_test,
        'y_test': y_test
    }


In [10]:
data_splits = split_and_save_data(
    input_df=modelling_data,
    target_col="target",
    processed_dir=processed_dir,
    random_state=random_state_var, 
)
X_train = data_splits['X_train']
y_train = data_splits['y_train']
X_valid = data_splits['X_valid']
y_valid = data_splits['y_valid']
X_test = data_splits['X_test']
y_test = data_splits['y_test']

print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"X_valid.shape: {X_valid.shape}")
print(f"y_valid.shape: {y_valid.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_test.shape: {y_test.shape}")

X_train.shape: (3017, 78)
y_train.shape: (3017,)
X_valid.shape: (646, 78)
y_valid.shape: (646,)
X_test.shape: (647, 78)
y_test.shape: (647,)


In [13]:
X_train.head(2)

,Age,YearsAtCompany,MonthlyIncome,DistanceFromHome,PercentSalaryHike,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,WorkLifeBalanceScore_dummy,RoleToCompanyRatio,...,MaritalGenderInteraction_Single_Female,MaritalGenderInteraction_Single_Male,OvertimeJobSatisfactionInteraction_No_1.0,OvertimeJobSatisfactionInteraction_No_2.0,OvertimeJobSatisfactionInteraction_No_3.0,OvertimeJobSatisfactionInteraction_No_4.0,OvertimeJobSatisfactionInteraction_Yes_1.0,OvertimeJobSatisfactionInteraction_Yes_2.0,OvertimeJobSatisfactionInteraction_Yes_3.0,OvertimeJobSatisfactionInteraction_Yes_4.0
1656,-1.194406,-1.429174,-1.712325,-0.553795,1.508009,-1.007268,-0.918712,-0.927282,-0.704830,-0.427709,...,False,False,False,False,False,False,False,True,False,False
2629,-0.751392,0.302370,-0.791332,0.364607,0.958808,-0.304356,-0.415772,-0.676854,-1.405271,-0.622058,...,False,False,False,False,False,False,False,True,False,False


In [14]:
y_train.head()

1656    1
2629    0
4041    1
370     1
4168    1
Name: target, dtype: int64

In [80]:
# src/model.py
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import joblib
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import joblib
import pandas as pd
import mlflow
# Define preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer



In [40]:

def get_model_and_params(model_name, random_state=random_state_var):
    if model_name == 'RandomForest':
        model = RandomForestClassifier(random_state=random_state, class_weight='balanced')
        param_distributions = {
            'classifier__n_estimators': [100, 200, 300, 400, 500],
            'classifier__max_depth': [None, 10, 20, 30, 40, 50],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4],
            'classifier__max_features': ['sqrt', 'log2']
        }
    elif model_name == 'XGBoost':
        model = XGBClassifier(random_state=random_state, eval_metric='logloss')
        param_distributions = {
            'classifier__n_estimators': [100, 200, 300, 400, 500],
            'classifier__max_depth': [3, 5, 7, 9, 11],
            'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
            'classifier__subsample': [0.6, 0.8, 1.0],
            'classifier__colsample_bytree': [0.6, 0.8, 1.0]
        }
    else:
        raise ValueError("Unsupported model name")
    
    return model, param_distributions


def train_model(model_name, X_train, y_train, preprocessing_pipeline, param_distributions, cv=5, n_iter=50, random_state=random_state_var):
    # Start an MLflow run for the specific model
    with mlflow.start_run(run_name=model_name) as run:
        mlflow.log_param("model", model_name)
        mlflow.log_param("cv", cv)
        mlflow.log_param("n_iter", n_iter)
        mlflow.log_param("random_state", random_state)
        
        # Get the model and its parameter grid
        classifier, param_dist = get_model_and_params(model_name, random_state)
        
        # Create a pipeline with preprocessing and model
        pipeline = Pipeline(steps=[
            # ('preprocessing', preprocessing_pipeline),
            ('classifier', classifier)
        ])
        
        # Initialize RandomizedSearchCV
        search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_dist,
            n_iter=n_iter,
            scoring='roc_auc',
            cv=cv,
            verbose=2,
            
            random_state=random_state,
            n_jobs=-1
        )
        
        # Fit the model
        search.fit(X_train, y_train)
        
        # Log the best parameters and score
        mlflow.log_params(search.best_params_)
        mlflow.log_metric("best_score", search.best_score_)
        
        # Log the best model
        mlflow.sklearn.log_model(search.best_estimator_, "best_model")
        
        return search
    

def evaluate_model(model, X, y, model_name="Model", dataset_name="validation"):
    with mlflow.start_run(run_name=f"Evaluation_{model_name}_{dataset_name}", nested=True):
        # Predictions
        y_pred = model.predict(X)
        y_proba = model.predict_proba(X)[:,1]
        
        # Evaluation Metrics
        report = classification_report(y, y_pred, output_dict=True)
        roc_auc = roc_auc_score(y, y_proba)
        conf_matrix = confusion_matrix(y, y_pred)
        
        # Log metrics
        mlflow.log_metric("roc_auc", roc_auc)
        mlflow.log_metrics({
            "precision": report['1']['precision'],
            "recall": report['1']['recall'],
            "f1_score": report['1']['f1-score']
        })
        
        # Log confusion matrix as an artifact
        conf_matrix_df = pd.DataFrame(conf_matrix, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
        conf_matrix_df.to_csv('confusion_matrix.csv')
        mlflow.log_artifact('confusion_matrix.csv')
        
        # Save evaluation metrics
        eval_metrics = {
            'classification_report': report,
            'roc_auc_score': roc_auc,
            'confusion_matrix': conf_matrix.tolist()
        }
        return eval_metrics

In [45]:
# Set the maximum number of rows to display
pd.set_option('display.max_rows', 5)



In [44]:
# Models to train
models_to_train = ['XGBoost', 'RandomForest']
evaluation_results = {}

preprocessing = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

for model_name in models_to_train:
    # Train the model with hyperparameter tuning
    best_search = train_model(
        model_name=model_name,
        X_train=X_train,
        y_train=y_train,
        preprocessing_pipeline=preprocessing,
        param_distributions=None,  # Already handled in train_model
        cv=5,
        n_iter=50,
        random_state=random_state_var
    )
    
    # Retrieve the best model
    best_model = best_search.best_estimator_
    
    # Save the best model using joblib
    model_filename = f'best_model_{model_name.lower()}.pkl'
    joblib.dump(best_model, os.path.join(models_dir, model_filename))
    mlflow.log_artifact(os.path.join(models_dir, model_filename))
    
    # Evaluate on test set
    eval_metrics_test = evaluate_model(best_model, X_test, y_test, model_name=model_name, dataset_name="test")
    
    # Log evaluation metrics
    evaluation_results[model_name] = eval_metrics_test
    
    # Prepare dashboard data
    # feature_importance_df, metrics_df = prepare_dashboard_data(model_name, best_model, X_test, y_test)
    
    # # Log feature importance and metrics to MLflow
    # mlflow.log_artifact(f'reports/feature_importance_{model_name.lower()}.png')
    # mlflow.log_artifact(f'reports/shap_summary_{model_name.lower()}.png')
    # mlflow.log_artifact(f'reports/dashboard_feature_importance_{model_name.lower()}.csv')
    # mlflow.log_artifact(f'reports/dashboard_metrics_{model_name.lower()}.csv')
    
    mlflow.end_run()


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:23:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:23:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:23:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgbo

[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1.0; total time=   0.2s
[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1.0; total time=   0.2s
[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1.0; total time=   0.2s
[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1.0; total time=   0.1s


/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:23:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:23:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:23:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgbo

[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=100, classifier__subsample=1.0; total time=   0.2s
[CV] END classifier__colsample_bytree=0.6, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.6; total time=   0.4s
[CV] END classifier__colsample_bytree=0.6, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.6; total time=   0.3s
[CV] END classifier__colsample_bytree=0.6, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.6; total time=   0.3s


...

/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:24:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:24:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:24:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.8; total time=   0.3s
[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.8; total time=   0.3s
[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.8; total time=   0.3s
[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.8; total time=   0.3s
[CV] END classifier__colsample_bytree=1.0, classifier__learning_rate=0.2, classifier__max_depth=3, classifier__n_estimators=300, classifier__subsample=0.8; total time=   0.2s
[CV] END classifier__colsample_bytree=0.8, classifier__learning_rate=0.1, classifier__max_depth=7, classifier__n_estimators=5

/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:24:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2024/09/17 19:24:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ryanmccarlie/.pyenv/versions/3.11.6/envs/PwC-Attrition-model-3.11/lib/python3.11/site-packages/sklearn/metrics/_classifi

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END classifier__max_depth=None, classifier__max_features=log2, classifier__min_samples_leaf=2, classifier__min_samples_split=10, classifier__n_estimators=400; total time=   1.6s
[CV] END classifier__max_depth=None, classifier__max_features=log2, classifier__min_samples_leaf=2, classifier__min_samples_split=10, classifier__n_estimators=400; total time=   1.6s
[CV] END classifier__max_depth=None, classifier__max_features=log2, classifier__min_samples_leaf=2, classifier__min_samples_split=10, classifier__n_estimators=400; total time=   1.7s
[CV] END classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=500; total time=   2.4s
[CV] END classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=500; total time=   2.4s
[CV] END classifier__max_dept

2024/09/17 19:25:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [58]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import mlflow
from sklearn.metrics import roc_auc_score, classification_report


def plot_feature_importance(model, feature_names, top_n=20, save_path='reports/feature_importance.png'):
    # Extract feature importances
    importances = model.named_steps['classifier'].feature_importances_
    feature_importance = pd.Series(importances, index=feature_names)
    top_features = feature_importance.sort_values(ascending=False).head(top_n)
    
    # Plot
    plt.figure(figsize=(10,8))
    sns.barplot(x=top_features.values, y=top_features.index)
    plt.title('Top Feature Importances')
    plt.xlabel('Importance Score')
    plt.ylabel('Features')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    
    # Log the plot as an MLflow artifact
    # mlflow.log_artifact(save_path)
    

# import shap
# def shap_explain(model, X, save_path='reports/shap_summary.png'):
#     # Initialize SHAP explainer
#     # Apply preprocessing pipeline
#     X_transformed = model.named_steps['preprocessing'].transform(X)
#     if hasattr(model.named_steps['classifier'], 'get_booster'):  # XGBoost specific
#         explainer = shap.Explainer(model.named_steps['classifier'])
#         shap_values = explainer(X_transformed)
#     else:
#         explainer = shap.Explainer(model.named_steps['classifier'], X_transformed)
#         shap_values = explainer(X_transformed)
    
#     # Summary plot
#     plt.figure(figsize=(10, 8))
#     shap.summary_plot(shap_values, X, show=False)
#     plt.savefig(save_path, bbox_inches='tight')
#     plt.close()
    
#     # Log the SHAP summary plot
#     # mlflow.log_artifact(save_path)

In [64]:
def prepare_dashboard_data(model_name, model, X, y):
    # Feature Importance
    fi_png_path = os.path.join(report_dir, f"feature_importance_{model_name.lower()}.png")
    plot_feature_importance(model, X.columns, top_n=20, save_path=fi_png_path)
    
    # SHAP Summary Plot
    # shap_explain(model, X, save_path=f'{save_path_prefix}shap_summary_{model_name.lower()}.png')
    
    # Prepare dashboard metrics
    importances = model.named_steps['classifier'].feature_importances_
    feature_names = X.columns
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)
    
    # Model performance metrics
    y_proba = model.predict_proba(X)[:,1]
    roc_auc = roc_auc_score(y, y_proba)
    report = classification_report(y, model.predict(X), output_dict=True)
    
    # Save feature importance
    feat_imp_out = os.path.join(report_dir, f"dashboard_feature_importance_{model_name.lower()}.csv")
    feature_importance_df.to_csv(feat_imp_out, index=False)
    
    # Save metrics
    metrics = {
        'Model': model_name,
        'ROC_AUC': roc_auc,
        'Precision': report['1']['precision'],
        'Recall': report['1']['recall'],
        'F1_Score': report['1']['f1-score']
    }
    metrics_df = pd.DataFrame([metrics])
    dm_csv_path = os.path.join(report_dir, f"dashboard_metrics_{model_name.lower()}.csv")
    metrics_df.to_csv(dm_csv_path, index=False)
    
    return feature_importance_df, metrics_df

In [65]:
feature_importance_df, metrics_df = prepare_dashboard_data(model_name, best_model, X_test, y_test)

In [76]:
for model, metrics in evaluation_results.items():
    break

metrics

{'classification_report': {'0': {'precision': 0.633693972179289,
   'recall': 1.0,
   'f1-score': 0.7757805108798487,
   'support': 410.0},
  '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 237.0},
  'accuracy': 0.633693972179289,
  'macro avg': {'precision': 0.3168469860896445,
   'recall': 0.5,
   'f1-score': 0.38789025543992434,
   'support': 647.0},
  'weighted avg': {'precision': 0.4015680503763655,
   'recall': 0.633693972179289,
   'f1-score': 0.4916074334787294,
   'support': 647.0}},
 'roc_auc_score': np.float64(0.6181022949470001),
 'confusion_matrix': [[410, 0], [237, 0]]}

In [78]:
# Save all evaluation metrics to a JSON file
import json
json_out = os.path.join(report_dir, 'evaluation_metrics.json')
with open(json_out, 'w') as f:
    json.dump(evaluation_results, f, indent=4)
# mlflow.log_artifact('reports/evaluation_metrics.json')

# Optionally, save a summary report
summary_df = pd.DataFrame([
    {
        'Model': model,
        'ROC_AUC': metrics['roc_auc_score'],
        'Precision': metrics['classification_report']['0']['precision'],
        'Recall': metrics['classification_report']['0']['recall'],
        'F1_Score': metrics['classification_report']['0']['f1-score']
    } for model, metrics in evaluation_results.items()
])
summ_fname = os.path.join(report_dir, "model_comparison.csv")
summary_df.to_csv(summ_fname, index=False)

# mlflow.log_artifact('reports/model_comparison.csv')

# Print evaluation scores for test set
print("\n=== Model Evaluation on Test Set ===")
for model, metrics in evaluation_results.items():
    print(f"\nModel: {model}")
    print(f"ROC AUC Score: {metrics['roc_auc_score']:.4f}")
    print("Classification Report:")
    print(pd.DataFrame(metrics['classification_report']).transpose())
    print("Confusion Matrix:")
    print(pd.DataFrame(metrics['confusion_matrix'], 
                        index=['Actual 0', 'Actual 1'], 
                        columns=['Predicted 0', 'Predicted 1']))

print("\nAll models have been trained, evaluated, and logged to MLflow.")


=== Model Evaluation on Test Set ===

Model: XGBoost
ROC AUC Score: 0.6181
Classification Report:
              precision    recall  f1-score     support
0              0.633694  1.000000  0.775781  410.000000
1              0.000000  0.000000  0.000000  237.000000
accuracy       0.633694  0.633694  0.633694    0.633694
macro avg      0.316847  0.500000  0.387890  647.000000
weighted avg   0.401568  0.633694  0.491607  647.000000
Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0          410            0
Actual 1          237            0

Model: RandomForest
ROC AUC Score: 0.5767
Classification Report:
              precision    recall  f1-score     support
0              0.656250  0.870732  0.748428  410.000000
1              0.485437  0.210970  0.294118  237.000000
accuracy       0.629057  0.629057  0.629057    0.629057
macro avg      0.570843  0.540851  0.521273  647.000000
weighted avg   0.593680  0.629057  0.582011  647.000000
Confusion Matrix:
          Predicted 0 

In [ ]:
mlflow.end_run()

## Generate Dashboard

In [ ]:
# src/mlflow_integration.py
import mlflow
from src.data_preprocessing import split_and_save_data
from src.model import train_model
from src.evaluate_model import evaluate_model
from src.visualize import plot_feature_importance, shap_explain
import pandas as pd
import joblib


# Start a top-level MLflow run
with mlflow.start_run(run_name="Employee Attrition Pipeline") as run:
    # Data Splitting
    data_splits = split_and_save_data()
    X_train = data_splits['X_train']
    y_train = data_splits['y_train']
    X_valid = data_splits['X_valid']
    y_valid = data_splits['y_valid']
    X_test = data_splits['X_test']
    y_test = data_splits['y_test']
    
    # Define preprocessing pipeline
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    
    preprocessing = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median'))
    ])
    
    # Define hyperparameter grid
    param_distributions = {
        'classifier__n_estimators': [100, 200, 300, 400, 500],
        'classifier__max_depth': [None, 10, 20, 30, 40, 50],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__max_features': ['auto', 'sqrt', 'log2']
    }
    
    # Train the model with hyperparameter tuning
    best_search = train_model(
        X_train, y_train, preprocessing, param_distributions
    )
    
    # Retrieve the best model
    best_model = best_search.best_estimator_
    
    # Log the model as an artifact in the top-level run
    mlflow.sklearn.log_model(best_model, "best_model_pipeline")
    
    # Evaluate on validation set
    eval_metrics_valid = evaluate_model(best_model, X_valid, y_valid, dataset_name="validation")
    
    # Evaluate on test set
    eval_metrics_test = evaluate_model(best_model, X_test, y_test, dataset_name="test")
    
    # Feature Importance
    plot_feature_importance(best_model, X_train.columns, save_path='reports/feature_importance.png')
    
    # SHAP Summary Plot
    # shap_explain(best_model, X_valid, save_path='reports/shap_summary.png')
    
    # Log evaluation metrics as artifacts
    import json
    with open('reports/evaluation_metrics.json', 'w') as f:
        json.dump({
            'validation': eval_metrics_valid,
            'test': eval_metrics_test
        }, f, indent=4)
    mlflow.log_artifact('reports/evaluation_metrics.json')
    
    # Log feature importance and SHAP plots as artifacts
    mlflow.log_artifact('reports/feature_importance.png')
    mlflow.log_artifact('reports/shap_summary.png')
    
    # Save Dashboard Data
    def prepare_dashboard_data(model, X, y, save_path='reports/dashboard_metrics.csv'):
        importances = model.named_steps['classifier'].feature_importances_
        feature_names = X.columns
        feature_importance = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values(by='Importance', ascending=False)
        
        # Add model performance metrics
        from sklearn.metrics import roc_auc_score, classification_report
        y_proba = model.predict_proba(X)[:,1]
        roc_auc = roc_auc_score(y, y_proba)
        report = classification_report(y, model.predict(X), output_dict=True)
        
        # Save feature importance
        feature_importance.to_csv('reports/dashboard_feature_importance.csv', index=False)
        mlflow.log_artifact('reports/dashboard_feature_importance.csv')
        
        # Save metrics
        metrics = {
            'ROC_AUC': roc_auc,
            'Precision': report['1']['precision'],
            'Recall': report['1']['recall'],
            'F1_Score': report['1']['f1-score']
        }
        pd.DataFrame([metrics]).to_csv(save_path, index=False)
        mlflow.log_artifact(save_path)
    
    prepare_dashboard_data(best_model, X_valid, y_valid)
    
    # Optionally, save the model using joblib
    joblib.dump(best_model, 'models/best_model.pkl')
    mlflow.log_artifact('models/best_model.pkl')
